--Copy Table script (need to delete table first)  
SELECT * INTO dbo.chocolate_data_expand  
FROM dbo.chocolate_data;

Questions:

Which factors give out the highest average rating (cocoa%, region, ingredients)?

How do unique characteristics play into ratings?

<span style="color: var(--vscode-foreground);">The table consists of 7 columns:</span>

- Company - Name of Manufacturer
- Company\_Location - ex. U.S.A., France, Japan, etc.
- Country\_of\_Bean\_Origin - ex. India, Vietnam, etc.
- Cocoa\_Percent - How much of the chocolate bar is made from cocoa
- Ingredients - List of ingredients in the bar where 
    - B = Beans
    - S = Sugar
    - S\* = Sweetener other than white cane or beet sugar
    - C = Cocoa Butter
    - V = Vanilla
    - L = Lecithin
    - Sa = Salt
- Most\_Memorable\_Characteristics - List of characteristics memorable to the chocolate
- Rating - Score out of 5 the reviewers gave it

<span style="color: var(--vscode-foreground);">After scraping&nbsp;</span>  <span style="color: var(--vscode-foreground);">the data from python a</span><span style="color: var(--vscode-foreground);">nd importing it to Azure Data Studio, we need to do some initial cleaning. Because I don't want to double count rows with the splitting of Ingredients/Characteristics later on, I will store these changes in an '_expand' table different from the original table.</span> 

For the Ingredients/Characteristics column, some ingredients have spaces between commas and some characteristics are quite lengthy, so we'll remove commas and spaces to shorten them.

For the Ingredients column especially, there's a text indicator (ex. 3- B,C,S) that we need to remove from our table.

In [ ]:
UPDATE dbo.chocolate_data_expand
SET Ingredients = right(Ingredients, len(Ingredients) - charindex('-', Ingredients))
UPDATE dbo.chocolate_data_expand
SET Ingredients = REPLACE(Ingredients, CHAR(32), ''),
    Most_Memorable_Characteristics = REPLACE(Most_Memorable_Characteristics, CHAR(32), '')
--SELECT TOP 3 * FROM dbo.chocolate_data_expand

Now we can do some EDA to answer potential questions on column values which result in higher ratings for our critics: Cocoa %, Company Location, and finally Ingredients/Characteristics.

In [ ]:
--Finding average rating of cocoa%
SELECT Cocoa_Percent, Count(*) AS [Num_Records], AVG(Rating) AS [Average_Rating]
FROM dbo.chocolate_data
GROUP BY Cocoa_Percent
HAVING Count(*) > 1
ORDER BY AVG(Rating) desc;

Highest rated cocoa percentage is between 66%-69%, with a decent sample size.

In [ ]:
--Finding average rating by Company_Location, at least 5 records
SELECT Company_Location, Count(*) AS [Num_Records], AVG(Rating) AS [Average_Rating]
FROM dbo.chocolate_data
GROUP BY Company_Location
HAVING Count(*) >= 5
ORDER BY AVG(Rating) desc;

USA holds the most observations of chocolate bars with 1227, followed by Canada/France with ~186 each. The top average ratings spot goes to U.A.E. with 3.4 followed by Poland and Denmark.

Now let's look at the Ingredients breakdown.

In [ ]:
--Finding average rating of ingredients
SELECT TRIM([value]) as split_ingredients, AVG(Rating) AS [Average_Rating], COUNT([value])
FROM dbo.chocolate_data_expand
    CROSS APPLY string_split(Ingredients, ',')
GROUP BY [value]
ORDER BY Average_Rating DESC;

Sugar has the highest average rating and non-sugar sweetener lowest. Honestly, not surprising considering the human palate likes sugary items which also makes other sweeteners taste different. This is only an observation and does not take into account interactions of ingredients, which can be explored at a different time.

And finally, characteristics.

In [ ]:
--Finding average rating of characteristics
SELECT TRIM([value]) as split_characteristics, AVG(Rating) AS [Average_Rating], COUNT([value]) AS [CNT]
FROM dbo.chocolate_data_expand
    CROSS APPLY string_split(Most_Memorable_Characteristics, ',')
GROUP BY [value]
ORDER BY Average_Rating DESC;

Doing a quick contextual analysis, pleasant-sounding and unique characteristics yield top rated chocolates.

In [ ]:
SELECT TRIM([value]) as split_characteristics, AVG(Rating) AS [Average_Rating], COUNT([value]) AS [CNT]
FROM dbo.chocolate_data_expand
    CROSS APPLY string_split(Most_Memorable_Characteristics, ',')
GROUP BY [value]
HAVING COUNT([value]) > 10
ORDER BY Average_Rating DESC;

For more common chocolates, balanced and strong fruity flavors reign supreme.

There were also a lot of unique characteristics of chocolates that merits further exploration.

In [ ]:
--Exploring unique characteristics
SELECT *
FROM dbo.chocolate_data_expand
    CROSS APPLY string_split(Most_Memorable_Characteristics, ',')
WHERE value = 'longandrich';
--We can see this unique characteristic comes from Belgium,
--how many of these one-off ones come from there or other places?
WITH Choco_CTE
AS
(
    SELECT *
    FROM dbo.chocolate_data_expand
        CROSS APPLY string_split(Most_Memorable_Characteristics, ',')
),
Choco_CTE_2
AS
(
    SELECT value, AVG(Rating) AS [Average_Rating], COUNT([value]) AS [CNT]
    FROM Choco_CTE
    GROUP BY [value]
),
Choco_CTE_3
AS
(
    SELECT *
    FROM Choco_CTE_2
    WHERE CNT = 1
)
-- To find all rows with unique characteristics
SELECT *
FROM Choco_CTE_3
JOIN Choco_CTE ON Choco_CTE_3.value = Choco_CTE.value;
/* SELECT Company, Company_Location, AVG(Rating) AS average, COUNT(Company)  
FROM Choco_CTE_3  
JOIN Choco_CTE ON Choco_CTE_3.value = Choco_CTE.value  
GROUP BY Company, Company_Location  
ORDER BY average DESC; */
/* SELECT Company_Location, COUNT(Company_Location)  
FROM Choco_CTE_3  
JOIN Choco_CTE ON Choco_CTE_3.value = Choco_CTE.value  
GROUP BY Company_Location  
ORDER BY COUNT(Company_Location) DESC; */

It seems U.S.A. has lots of top rated unique chocolate characteristics, but probably due to the large quantity of them inflating their numbers. <span style="color: var(--vscode-foreground);">Surprisingly, Chile and Vietnam who have 1 and 2 chocolate entries respectively, hold top 10 spots.</span>

Notes for future project: Generate primary key for each entry to not double count rows when analyzing ingredients/characteristics